In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#한글폰트 가져오기
from matplotlib import rc
rc('font', family='NanumGothic')

# Warning 메세지를 뜨지 않게 해줌
import warnings
warnings.filterwarnings('ignore')

import os 
os.chdir("C:/Users/sim/Desktop/medical/data/")
currentpath = os.getcwd()
currentpath

'C:\\Users\\sim\\Desktop\\medical\\data'

In [24]:
df = pd.read_csv('Final.csv')
df_origin = df.copy()

In [25]:
df

,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,...,감마지티피,흡연상태,음주여부,시력,청력,신장(m),BMI,WHtR,BMI_WC_risk,WHtR_risk
0,2,5,170,50,69.2,92.0,60.0,93.0,12.2,1.0,...,12.0,1.0,1.0,1.10,1.0,1.70,17.301038,0.407059,0,0
1,1,6,170,75,91.0,120.0,69.0,92.0,14.9,1.0,...,16.0,1.0,1.0,0.85,1.0,1.70,25.951557,0.535294,4,2
2,2,6,155,55,74.4,109.0,67.0,90.0,13.7,1.0,...,12.0,1.0,1.0,1.20,1.0,1.55,22.892820,0.480000,1,1
3,2,5,165,70,79.0,152.0,91.0,114.0,13.7,1.0,...,19.0,1.0,1.0,1.00,1.0,1.65,25.711662,0.478788,3,1
4,1,5,175,75,76.0,110.0,70.0,96.0,14.9,1.0,...,24.0,3.0,1.0,1.35,1.0,1.75,24.489796,0.434286,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104814,2,5,165,70,74.0,126.0,79.0,85.0,11.2,1.0,...,22.0,3.0,1.0,1.10,1.0,1.65,25.711662,0.448485,3,1
104815,1,6,175,75,85.0,121.0,68.0,97.0,16.0,1.0,...,132.0,1.0,1.0,1.20,1.0,1.75,24.489796,0.485714,2,1
104816,2,6,160,50,62.0,100.0,61.0,93.0,13.6,1.0,...,12.0,1.0,1.0,1.10,1.0,1.60,19.531250,0.387500,1,0
104817,2,6,160,50,65.0,98.0,67.0,96.0,13.2,3.0,...,25.0,1.0,0.0,1.20,1.0,1.60,19.531250,0.406250,1,0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104819 entries, 0 to 104818
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   성별코드         104819 non-null  int64  
 1   연령대코드(5세단위)  104819 non-null  int64  
 2   신장(5cm단위)    104819 non-null  int64  
 3   체중(5kg단위)    104819 non-null  int64  
 4   허리둘레         104819 non-null  float64
 5   수축기혈압        104819 non-null  float64
 6   이완기혈압        104819 non-null  float64
 7   식전혈당(공복혈당)   104819 non-null  float64
 8   혈색소          104819 non-null  float64
 9   요단백          104819 non-null  float64
 10  혈청크레아티닌      104819 non-null  float64
 11  혈청지오티(AST)   104819 non-null  float64
 12  혈청지피티(ALT)   104819 non-null  float64
 13  감마지티피        104819 non-null  float64
 14  흡연상태         104819 non-null  float64
 15  음주여부         104819 non-null  float64
 16  시력           104819 non-null  float64
 17  청력           104819 non-null  float64
 18  신장(m)        104819 non-

In [27]:
df.shape

(104819, 23)

In [28]:
df['연령대코드(5세단위)'].unique()

array([5, 6], dtype=int64)

In [29]:
df['연령대코드(5세단위)'].value_counts().sort_index()

연령대코드(5세단위)
5    33930
6    70889
Name: count, dtype: int64

In [30]:
df = df.drop(['BMI', 'WHtR', '연령대코드(5세단위)', '신장(5cm단위)', 
              '체중(5kg단위)','허리둘레', '신장(m)'], axis=1)

## 가경이행, 철수행 생성

- 기준 : 질병관리청 국가건강정보포털, 서울아산병원
- 아래 변수에 계산되어 포함되어 있는 변수(신장, 체중, 허리둘레 등)은 제외
- 건강한가경이행과 건강한철수행을 기준으로 잡아 각 변수별 데이터마다 유사도를 책정하여 진행할 예정
- '완벽행' 열 생성 >> 기준이 되는 가경이행, 철수행과 나머지 데이터를 구분


|**변수**|**건강한가경이행**|**건강한철수행**|
|------|---|---|
|**성별코드**|2|1|
|**수축기혈압**|105|105|
|**이완기혈압**|70|70|
|**식전혈당(공복혈당)**|90|90|
|**혈색소**|12.75|14.75|
|**요단백**|1|1|
|**혈청크레아티닌**|1.25|1.25|
|**혈청지오티(AST)**|20|20|
|**혈청지피티(ALT)**|17.5|17.5|
|**감마지티피**|21.5|37|
|**흡연상태**|1|1|
|**음주여부**|0|0|
|**시력**|0.6|0.6|
|**청력**|1|1|
|**BMI_WC_risk**|1|1|
|**WHtR_risk**|1|1|

In [31]:
new_data = {
    '성별코드': [2, 1],  # 가경이 여자, 철수 남자
    '수축기혈압': [105, 105],
    '이완기혈압': [70, 70],
    '식전혈당(공복혈당)': [90, 90],
    '혈색소': [13.75, 14.75],
    '요단백': [1, 1], # 음성이 정상
    '혈청크레아티닌': [1.25, 1.25],
    '혈청지오티(AST)': [20, 20],
    '혈청지피티(ALT)': [17.5, 17.5],
    '감마지티피': [21.5, 37],
    '흡연상태': [1, 1],
    '음주여부': [0, 0],
    '시력': [0.6, 0.6],
    '청력': [1, 1],
    'BMI_WC_risk': [1, 1],
    'WHtR_risk': [1, 1] 
}

# 새로운 행을 데이터프레임으로 변환
new_rows_df = pd.DataFrame(new_data)

# 기준행 더미변수 추가
new_rows_df['완벽행'] = 1  # 기준 행에는 1로 설정

# 기존 df와 새로운 행을 병합
df = pd.concat([df, new_rows_df], ignore_index=True)

# 나머지 행에는 0으로 설정
df['완벽행'].fillna(0, inplace=True)

# 결과 확인
df.tail()

,성별코드,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk,완벽행
104816,2,100.0,61.0,93.0,13.60,1.0,0.80,20.0,14.0,12.0,1.0,1.0,1.1,1.0,1,0,0.0
104817,2,98.0,67.0,96.0,13.20,3.0,1.00,21.0,14.0,25.0,1.0,0.0,1.2,1.0,1,0,0.0
104818,1,125.0,80.0,92.0,16.90,1.0,1.10,30.0,20.0,16.0,1.0,1.0,0.8,1.0,2,0,0.0
104819,2,105.0,70.0,90.0,13.75,1.0,1.25,20.0,17.5,21.5,1.0,0.0,0.6,1.0,1,1,1.0
104820,1,105.0,70.0,90.0,14.75,1.0,1.25,20.0,17.5,37.0,1.0,0.0,0.6,1.0,1,1,1.0


In [32]:
df['완벽행'] = df['완벽행'].astype(int)

In [33]:
df # 104819 + 2(완벽행 두개)

,성별코드,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk,완벽행
0,2,92.0,60.0,93.0,12.20,1.0,0.60,16.0,11.0,12.0,1.0,1.0,1.10,1.0,0,0,0
1,1,120.0,69.0,92.0,14.90,1.0,0.70,31.0,48.0,16.0,1.0,1.0,0.85,1.0,4,2,0
2,2,109.0,67.0,90.0,13.70,1.0,0.60,12.0,8.0,12.0,1.0,1.0,1.20,1.0,1,1,0
3,2,152.0,91.0,114.0,13.70,1.0,0.70,21.0,25.0,19.0,1.0,1.0,1.00,1.0,3,1,0
4,1,110.0,70.0,96.0,14.90,1.0,1.20,28.0,21.0,24.0,3.0,1.0,1.35,1.0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104816,2,100.0,61.0,93.0,13.60,1.0,0.80,20.0,14.0,12.0,1.0,1.0,1.10,1.0,1,0,0
104817,2,98.0,67.0,96.0,13.20,3.0,1.00,21.0,14.0,25.0,1.0,0.0,1.20,1.0,1,0,0
104818,1,125.0,80.0,92.0,16.90,1.0,1.10,30.0,20.0,16.0,1.0,1.0,0.80,1.0,2,0,0
104819,2,105.0,70.0,90.0,13.75,1.0,1.25,20.0,17.5,21.5,1.0,0.0,0.60,1.0,1,1,1


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104821 entries, 0 to 104820
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   성별코드         104821 non-null  int64  
 1   수축기혈압        104821 non-null  float64
 2   이완기혈압        104821 non-null  float64
 3   식전혈당(공복혈당)   104821 non-null  float64
 4   혈색소          104821 non-null  float64
 5   요단백          104821 non-null  float64
 6   혈청크레아티닌      104821 non-null  float64
 7   혈청지오티(AST)   104821 non-null  float64
 8   혈청지피티(ALT)   104821 non-null  float64
 9   감마지티피        104821 non-null  float64
 10  흡연상태         104821 non-null  float64
 11  음주여부         104821 non-null  float64
 12  시력           104821 non-null  float64
 13  청력           104821 non-null  float64
 14  BMI_WC_risk  104821 non-null  int64  
 15  WHtR_risk    104821 non-null  int64  
 16  완벽행          104821 non-null  int32  
dtypes: float64(13), int32(1), int64(3)
memory usage: 13.2 MB
